## Lesson 1A - IMDB Subwords 8K with Single Layer LSTM

In [1]:
!pip install tensorflow==2.0.0-beta1

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

print(tf.__version__)

2.0.0-beta1


In [0]:
## Get the data

dataset, info = tfds.load('imdb_reviews/subwords8k',
                          with_info = True,
                          as_supervised = True)

train_dataset, test_dataset = dataset['train'], dataset['test']

In [0]:
tokenizer = info.features['text'].encoder

In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE,
                                           train_dataset.output_shapes)
test_dataset = test_dataset.padded_batch(BATCH_SIZE,
                                         test_dataset.output_shapes)

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [9]:
NUM_EPOCHS = 10

history = model.fit(train_dataset,
                    epochs = NUM_EPOCHS,
                    validation_data = test_dataset)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


391/391 [==============================] - 1877s 5s/step - loss: 0.5149 - accuracy: 0.7388 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
391/391 [==============================] - 1532s 4s/step - loss: 0.3274 - accuracy: 0.8711 - val_loss: 0.4149 - val_accuracy: 0.8246
Epoch 3/10
  6/391 [..............................] - ETA: 15:06 - loss: 0.3438 - accuracy: 0.8698

KeyboardInterrupt: ignored

### Interrupted execution as each epoch was taking half an hour to run

In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_' + string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_' + string])
  plt.show()

In [0]:
plot_graphs(history, 'accuracy')

In [0]:
plot_graphs(history, 'loss')